In [1]:
#本代碼所涉及的所有函數庫
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Dropout,LSTM
from keras import backend
from keras.optimizers import SGD
from pandas import read_csv
import tensorflow.keras
import tensorflow as tf
import csv
import math
import random
import operator
from scipy.ndimage import gaussian_filter1d
from scipy.signal import medfilt
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.layers.wrappers import Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import GRU
%matplotlib inline

Using TensorFlow backend.


# LSTM模型

In [2]:
def classdatapredeal(dirdata1,dirdata2,string):
    
#     df = pd.read_csv(filedir1)#
#     #print(df)
#     attribute=df.columns#獲取屬性值
    attribute_set=[]
    count_arribute=0#基因選擇後的屬性個數
    #print(attribute)
    i=0
    for record in string: 
        if (record=='1'):
            count_arribute+=1
            attribute_set.append(i)
        i=1+i
    print(count_arribute)
    data1 = []
    label1 = []
    data2 = []
    label2 = []
    
    df1 = pd.read_csv(dirdata1,usecols=attribute_set,header = None)
    print(df1)
    rows = csv.reader(df1)
    for index, row in df1.iterrows():
        data1.append(row)
        label1.append(0)
    print(len(data1))

    
    df2 = pd.read_csv(dirdata2,usecols=attribute_set,header = None)
    print(df2)
    rows = csv.reader(df2)
    for index, row in df2.iterrows():
        data2.append(row)
        label2.append(1)
    print(len(data2))
    
   
    # print(data1)
    data1 = np.array(data1)
    label1 = np.array(label1)
    #print(data1)
    data2 = np.array(data2)
    label2 = np.array(label2)
    

    data1 = preprocessing.scale(data1)
    data2 = preprocessing.scale(data2)
   
    #print(data1)
    #print(data1)
#     data1=medfilt(data1,3)
#     data1=gaussian_filter1d(data1,1.2)
#     data2=medfilt(data2,3)
#     data2=gaussian_filter1d(data2,1.2)
#     sc=MinMaxScaler(feature_range=(0,1))
#     data1=sc.fit_transform(data1)
#     data2=sc.fit_transform(data2)
    return  data1,label1,data2,label2

In [3]:
def datasplit(data,label):
    train_X,test_X, train_y, test_y = train_test_split(data,
                                                   label,
                                                   test_size = 0.4,
                                                   random_state = 77,shuffle=True)
    return train_X,test_X, train_y, test_y
    

In [4]:
def datasplit1(data,label):
    train_X,test_X, train_y, test_y = train_test_split(data,
                                                   label,
                                                   test_size = 0.3,
                                                   random_state = 42,shuffle=True)
    return train_X,test_X, train_y, test_y
    

In [5]:
#分類數據重組
def classdataReorganization(data1,label1,data2,label2,step):
    re_train_data = []
    re_train_label=[]
    for i in range(len(data1)-step):
        re_train_data.append(data1[i:i+step])
        re_train_label.append(label1[i+step])

    for i in range(len(data2)-step):
        re_train_data.append(data2[i:i+step])
        re_train_label.append(label2[i+step])
    
    
    All_data = np.array(re_train_data)
    All_label = np.array(re_train_label)
     # Label Onehot-encoding 
    All_label = np_utils.to_categorical(All_label)
    return All_data,All_label

In [6]:
def timeModel(X,step):
# 序列到序列堆叠式LSTM模型 
    #temp=X.shape[2]
    model=Sequential()
    model.add(LSTM(20,activation='relu',input_shape=(step,X.shape[2]),return_sequences=True))
    model.add(LSTM(20,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(2,activation = 'sigmoid'))
    model.compile(loss='mse', optimizer="Adam", metrics=['accuracy'])
    model.summary()
    return model

In [7]:
def timeModel1(X,step):
# 序列到序列堆叠式LSTM模型 

    model=Sequential()
    model.add(LSTM(50,activation='relu',input_shape=(step,X.shape[2]),return_sequences=True))
    model.add(LSTM(50,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(2,activation = 'sigmoid'))
    model.compile(loss='mse', optimizer="Adam", metrics=['accuracy'])
    model.summary()
    return model

# 基因演算法

In [8]:
class GA():
    def __init__(self, length, count):
        # 染色體長度
        self.length = length
        # 種群中的染色體數量
        self.count = count
        # 隨機生成初始種群
        self.population = self.gen_population(length, count)
        for i in range(len(self.population)):
            print("染色體：")
            print(i)
            print (self.population[i])
            print ('{0:b}'.format(self.population[i]))
#         #數據文路徑
#         self.filedir=filedir
    def evolve(self, retain_rate=0.2, random_select_rate=0.5, mutation_rate=0.01):
        """
        進化
        對當前一代種群依次進行選擇、交叉並生成新一代種群，然後對新一代種群進行變異
        """
        parents = self.selection(retain_rate, random_select_rate)
        self.crossover(parents)
        self.mutation(mutation_rate)
    def gen_chromosome(self, length):
        """
        隨機生成長度為length的染色體，每個基因的取值是0或1
        這裡用一個bit表示一個基因
        """
        chromosome = 0
        for i in range(length):
            # 這裡的或是整個二進位制比對的或|,比如10和1或下來的結果是11，  1001和101或的結果是1101，所以此處是個不斷隨機或出來的長度
            chromosome |= (1 << i) * random.randint(0, 1)  # 這裡每次左移， 隨機生成一個0或者1的數字，這樣不斷左移生成給定長度的二進位制染色體

        #print ('{0:b}'.format(chromosome))
        print("gen_chromosome pass")
        return chromosome
    def gen_population(self, length, count):
        """
        獲取初始種群（一個含有count個長度為length的染色體的列表）
        """
        po=[]
        j=0
        while j<=count:#保证染色体长度一致
            chromosome=0
            for i in range(length):
                    # 這裡的或是整個二進位制比對的或|,比如10和1或下來的結果是11，  1001和101或的結果是1101，所以此處是個不斷隨機或出來的長度
                    chromosome |= (1 << i) * random.randint(0, 1) 
                    #print('{0:b}'.format(chromosome))
            string='{0:b}'.format(chromosome)
            len_chromosome=len(string)
            if len_chromosome==length:
                po.append(chromosome)
                j+=1

            
        print("gen_population pass")
        return po
    def fitness(self, chromosome):
            string='{0:b}'.format(chromosome)
            print("所選擇的染色體：")
            print(string)
            #檔案路徑
            class_dirdata1='n01new.csv'
            class_dirdata2='y01new.csv'
            #特征篩選
            class_data1,class_label1,class_data2,class_label2=classdatapredeal(class_dirdata1,class_dirdata2,string)
            time_step=100#時間序列設置
            #數據和標籤設定
            class_All_data,class_All_label=classdataReorganization(class_data1,class_label1,class_data2,class_label2,time_step)
            #訓練集測試集分割
            train_X,test_X, train_y, test_y=datasplit(class_All_data,class_All_label)
            print(train_X.shape)
            #模型訓練
            timemodel=timeModel(train_X,time_step)
            history = timemodel.fit(train_X,train_y,epochs=30, validation_split=0.2,batch_size=400, verbose=1)
            score = timemodel.evaluate(test_X, test_y, verbose=0)
            
            print(score[1])
            print("fitness pass")
            return score[1]
    def selection(self, retain_rate, random_select_rate):
        """
        選擇
        先對適應度從大到小排序，選出存活的染色體
        再進行隨機選擇，選出適應度雖然小，但是倖存下來的個體
        """
        # 對適應度從大到小進行排序
        graded = [(self.fitness(chromosome), chromosome) for chromosome in self.population]  # 求出所有染色體的適應值列表
        
        print(graded)
        graded = [x[1] for x in sorted(graded, reverse=True)]  # 從小到大排序，形成新列表
        # 選出適應性強的染色體
        retain_length = int(len(graded) * retain_rate)  # 根據比例值  找到前。個適應性強的染色體 作為下一代的父母親
        parents = graded[:retain_length]
        print(parents)
        # 選出適應性不強，但是倖存的染色體        從後面中  隨機選取一比例   也放到父母親中
        for chromosome in graded[retain_length:]:
            if random.random() < random_select_rate:
                parents.append(chromosome)
        print(parents)
        print("selection pass")
        return parents
    def crossover(self, parents):
        """
        染色體的交叉、繁殖，生成新一代的種群
        """
        # 新出生的孩子，最終會被加入存活下來的父母之中，形成新一代的種群。
        children = []
        # 需要繁殖的孩子的量，差值，就是要交叉生成的目標數
        target_count = len(self.population) - len(parents)
        print("target_count:")
        print(target_count)
        # 開始根據需要的量進行繁殖
        while len(children) < target_count:
            male = random.randint(0, len(parents) - 1)
            female = random.randint(0, len(parents) - 1)
            if male != female:  # 父母親序號不能相同
                # 隨機選取交叉點
                cross_pos = random.randint(0, self.length)
                # 生成掩碼，方便位操作
                mask = 0
                for i in range(cross_pos):
                    mask |= (1 << i)
                male = parents[male]
                female = parents[female]
                # 孩子將獲得父親在交叉點前的基因和母親在交叉點後（包括交叉點）的基因
                child = ((male & mask) | (female & ~mask)) & ((1 << self.length) - 1)
                children.append(child)
        # 經過繁殖後，孩子和父母的數量與原始種群數量相等，在這裡可以更新種群。
        self.population = parents + children
        for i in range(len(self.population)):
            print("生成新一代的種群")
            print("染色體：")
            print(i)
            print (self.population[i])
            print ('{0:b}'.format(self.population[i]))
        print("crossover pass")
    def mutation(self, rate):
        """
        變異
        對種群中的所有個體，隨機改變某個個體中的某個基因
        """
        for i in range(len(self.population)):
            if random.random() < rate:
                j = random.randint(0, self.length - 1)
                self.population[i] ^= 1 << j
        for i in range(len(self.population)):
            print("變異後的種群")
            print("染色體：")
            print(i)
            print (self.population[i])
            print ('{0:b}'.format(self.population[i]))
        print("mutation pass")
    def decode(self, chromosome):
            """
            解碼染色體，將二進位制轉化為屬於[0, 9]的實數
            """
            print("decode pass")
            return chromosome * 9.0 / (2 ** self.length - 1)
            
    def result(self):
        """
        獲得當前代的最優值，這裡取的是函式取最大值時x的值。
        """
#         print (self.population[i])
#         print ('{0:b}'.format(self.population[i]))
        print("result pass")
        return '{0:b}'.format(self.population[0])

# 合併演算法

In [ ]:
#geni_string=ga.result()
geni_string="10011001101111100010000111111000001000100111111110101110011001110110101111001"
print (geni_string)

In [ ]:
dir="flow1.csv"
df = pd.read_csv(dir)
col=df.columns
i=0
data1=[]
for record in geni_string:
    if(record=='1'):
        data1.append(col[i])
    i+=1
print(len(data1))

In [ ]:
print("所选择之属性：")
data1

In [ ]:
class_dirdata1='flow1_2class.csv'
class_dirdata2='flow2_2class.csv'
#特征篩選
class_data1,class_label1,class_data2,class_label2=classdatapredeal(class_dirdata1,class_dirdata2,geni_string)
time_step=100#時間序列設置


In [ ]:
class_data1.shape

In [ ]:
class_label2

In [ ]:
#數據和標籤設定
class_All_data,class_All_label=classdataReorganization(class_data1,class_label1,class_data2,class_label2,time_step)
#訓練集測試集分割
train_X,test_X, train_y, test_y=datasplit(class_All_data,class_All_label)


In [ ]:
test_y[:3]

In [ ]:
#模型訓練
timemodel=timeModel1(train_X,time_step)
history = timemodel.fit(train_X,train_y,epochs=30, validation_split=0.2,batch_size=500, verbose=1)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('class model evaluate', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

class_epoch_list = list(range(1,31))
ax1.plot(class_epoch_list, history.history['acc'], label='class_Train Accuracy')
ax1.plot(class_epoch_list, history.history['val_acc'], label='class_Validation Accuracy')
ax1.set_xticks(np.arange(0, 31, 5))
ax1.set_ylabel('class_Accuracy Value')
ax1.set_xlabel('class_Epoch')
ax1.set_title('class_Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(class_epoch_list, history.history['loss'], label='class_Train Loss')
ax2.plot(class_epoch_list, history.history['val_loss'], label='class_Validation Loss')
ax2.set_xticks(np.arange(0, 31, 5))
ax2.set_ylabel('class_Loss Value')
ax2.set_xlabel('class_Epoch')
ax2.set_title('class_Loss')
l2 = ax2.legend(loc="best")

In [ ]:
#對分類區中的test進行預測
time_pre=timemodel.predict(test_X,verbose=2)

In [ ]:
test_y[:3]

In [ ]:
time_pre[:3]

In [ ]:
# calculate RMSE
rmse = sqrt(mean_squared_error(test_y, time_pre))
print('Test RMSE: %.3f' % rmse)

In [ ]:
Y_pred = np.argmax(time_pre,axis=1)
Y_test = np.argmax(test_y,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
mat = confusion_matrix(Y_test,Y_pred)
from matplotlib import pyplot as plt
import seaborn as sns

sns.set(font_scale=1.5)
sns.heatmap(mat,square=True,annot=True,fmt='d',cbar=False)

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.xlabel('pred label')
plt.ylabel('true label')

t = ['0','1']
print(mat)
print(classification_report(Y_test,Y_pred,target_names=t))

In [ ]:
plt.figure(figsize=(4,3))


# plt.subplot(3,1,1)
plt.plot(Y_test[:100], color = 'black', linewidth=1, label = 'True value')
plt.plot(time_pre[:100], color = 'red', label = 'Predicted')
plt.legend(frameon=False)
plt.ylabel("label forecast")
plt.xlabel("time")
plt.title("Predicted label (first 100)")